# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-19-events.csv', '/workspace/home/event_data/2018-11-11-events.csv', '/workspace/home/event_data/2018-11-20-events.csv', '/workspace/home/event_data/2018-11-27-events.csv', '/workspace/home/event_data/2018-11-24-events.csv', '/workspace/home/event_data/2018-11-12-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-14-events.csv', '/workspace/home/event_data/2018-11-26-events.csv', '/workspace/home/event_data/2018-11-13-events.csv', '/workspace/home/event_data/2018-11-03-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-23-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-06-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [50]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
# Create a Keyspace 
try:
    session.execute("\
        CREATE KEYSPACE IF NOT EXISTS useractivity\
        WITH REPLICATION = \
        {'class':'SimpleStrategy','replication_factor': 1} "
    )
except Exception as e:
    print(e)


#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('useractivity')
except exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [7]:
# create table for Query1 : retrieve artist, song title and song's length in the music app history 
# based on sessionId and itemInSession
# Partition key - sessionId column as it is where clause attribute
# Clustering column - iteminsession column as it make PK unique and provides sorting
query = "CREATE TABLE IF NOT EXISTS activity_by_sessionid_iteminsession"
query = query + "(sessionid INT, iteminsession INT, artist TEXT, song TEXT, song_len FLOAT, PRIMARY KEY((sessionid),iteminsession))"

try:
    session.execute(query)
except exception as e:
    print(e)
    
# create table for Query2: retreive name of artist, song (sorted by itemInSession) and user (first and last name) 
# based on userid and sessionid
# Partition key -  userid and sessionid column as it is where clause attribute
# Clustering column - iteminsession column as it make PK unique and provides sorting
query = "CREATE TABLE IF NOT EXISTS activity_by_userid_sessionid"
query = query + "(userid INT, sessionid INT, iteminsession INT, artist TEXT, song TEXT, first_name TEXT, last_name TEXT, PRIMARY KEY((userid,sessionid),iteminsession))"

try:
    session.execute(query)
except exception as e:
    print(e)
    
# create table for Query3: retreive user name (first and last name)
# based on song 
# Partition key -  song column as it is where clause attribute
# Clustering column - userid column as it make PK unique and provides sorting
query = "CREATE TABLE IF NOT EXISTS user_by_song"
query = query + "(song TEXT, userid INT, first_name TEXT, last_name TEXT, PRIMARY KEY((song),userid))"

try:
    session.execute(query)
except exception as e:
    print(e)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Retreive  artist, song title and song's length in the music app history that was heard during 
# sessionId = 338, and itemInSession = 4

query = "Select artist, song, song_len as song_length from activity_by_sessionid_iteminsession where sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)
                    

Empty DataFrame
Columns: []
Index: []


In [9]:
# Query 1: Inserting the data from CSV file to the Table activity_by_sessionid_iteminsession

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO activity_by_sessionid_iteminsession(sessionid, iteminsession, artist, song, song_len)"
        query = query + "values(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]), line[0], line[9], float(line[5])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Query 1:  Retreive  artist, song title and song's length in the music app history that was heard during 
# sessionId = 338, and itemInSession = 4

query = "Select artist, song, song_len as song_length from activity_by_sessionid_iteminsession where sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
print(df)
    
    

      artist                             song  song_length
0  Faithless  Music Matters (Mark Knight Dub)   495.307312


In [11]:
# Query 2: Retreive name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

query = "Select artist, song, first_name, last_name from activity_by_userid_sessionid where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
print(df)
                        

Empty DataFrame
Columns: []
Index: []


In [12]:
# Query 2: Inserting the data from CSV file to the Table activity_by_userid_sessionid
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO activity_by_userid_sessionid(userid, sessionid, iteminsession, artist, song, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]) )

In [13]:
# Query 2: Retreive name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

query = "Select artist, song, first_name, last_name from activity_by_userid_sessionid where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except exception as e:
    print(e)
 
df = pd.DataFrame(list(rows))
print(df)
                       

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


In [14]:
# Query 3: Retreive every user name (first and last) who listened to the song 'All Hands Against His Own'

query = "Select first_name, last_name from user_by_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)   
except exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)
                    

Empty DataFrame
Columns: []
Index: []


In [16]:
# Query 3: Inserting the data from CSV file to the Table user_by_song
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song(song, userid, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]) )

In [17]:
# Query 3: Retreive every user name (first and last) who listened to the song 'All Hands Against His Own'
query = "Select first_name, last_name from user_by_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)   
except exception as e:
    print(e)

df = pd.DataFrame(list(rows))
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


### Drop the tables before closing out the sessions

In [18]:
# Drop the table before closing out the sessions

query = "Drop table activity_by_sessionid_iteminsession"
try:
    session.execute(query)   
except exception as e:
    print(e)
    
query = "Drop table activity_by_userid_sessionid"
try:
    session.execute(query)   
except exception as e:
    print(e)
    
query = "Drop table user_by_song"
try:
    session.execute(query)   
except exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()